# [모듈 6.1] 세이지메이커 모델 스텝 개발 (SageMaker Model Building Pipeline 훈련 스텝)

이 노트북은 아래와 같은 목차로 진행 됩니다. 전체를 모두 실행시에 완료 시간은 약 5분-10분 소요 됩니다.

- 0. 세이지메이커 모델 개요 
- 1. 모델 빌딩 파이프라인에서 세이지 메이커 모델 생성 스텝 실행 
    
---

# 0.세이지메이커 모델 개요

## SageMaker 호스팅 서비스를 사용하여 모델 배포는 다음과 같은 3단계 프로세스

![hosting-architecture.png](img/hosting-architecture.png)

### (1) SageMaker 에서 모델 생성 (이 노트북에서 포함하는 내용임)
- 모델을 생성하여 모델 컴포넌트를 찾을 수 있는 위치를 SageMaker 에 알려줍니다.
    - 모델 컴포넌트: 
        - (1) 모델 결과물(아티펙트)이 저장되는 S3 경로 (model.tar.gz)
        - (2) 추론 코드를 포함하는 이미지에 대한 도커 컨테이너 레지스트리 경로

### (2) HTTPS endpoint에 대한 endpoint configuration 만들기
- 프로덕션 변형(Product Varaiant) 에서 하나 이상의 모델 이름과 SageMaker 에서 각 프로덕션 변형을 호스트하기 위해 시작할 ML 인스턴스를 지정
    - 프로덕션 변형의 중요 인자
        - Model Name: 위의 "(1)" 에서 생성한 세이지 메이커 모델 경로
        - InstanceType: 배포할 EC2 인스턴스 타입 (에: ml.m4.2xlarge)
        - InitialInstanceCount: EC2 인스턴스 갯수

### (3) HTTPS 엔드포인트 생성
- endpoint configuration을 SageMaker에 제공합니다. SageMaker는 ML 인스턴스를 시작하고 configuration에서 지정된 대로 모델을 배포합니다. endpoint 에서 추론을 받으려면 클라이언트 애플리케이션에서 SageMaker 런타임 HTTPS 엔드포인트로 요청을 전송합니다. API에 대한 자세한 정보는 InvokeEndpoint API를 참조하십시오.



### [참조] 세이지 메이커 훈련, 배포, 추론 아키텍쳐
아래 그림에서 "Deployment / hosting on ML compute instances" 와 "Endpoint" 부분이 위의 세이지 메이커 모델 배포 3단계를 
- 상세 사항은 여기를 참조 하세요. --> [Amazon SageMaker 에서 모델 배포](https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/how-it-works-deployment.html)

![sagemaker_architecture.png](img/sagemaker-architecture.png)
### 예시: 세이지 메이커 모델 생성 내용
- Container1: Image
    - inference code가 사용하는 추론 다커 이미지 입니다.
- Container1: Model data location
    - 훈련 알고리즘(예: 사용자 정의 알고리즘 혹은 세이지 메이커 내장 알고리즘)으로 훈련을 하고 난후에 S3에 저장된 모델 아티펙트 입니다.


![sagemaker-model.png](img/sagemaker-model.png)

# 1. 모델 빌딩 파이프라인에서 세이지 메이커 모델 생성 스텝 실행 
---



기본적인 세이지 메이커 정보를 가져옵니다.

In [1]:
import boto3
import sagemaker
import pandas as pd

sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 

### 추론 실행에 사용할 모델 정의 

모델을 사용하는 배치변환작업을 실행하기 위해 SageMaker Model을 생성합니다.

```python
model = Model(
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)
```

구체적으로, `TrainingStep` 인스턴스인 `step_train`의 속성 중 `TrainingImagea, S3ModelArtifacts` 의 결과물을 입력으로 다시 사용합니다.
- 아래의 예시는 '세이지 메이커 모델 생성' 스텝을 독립적으로 실행하기 위해서, 이전 노트북의 결과인 image_uri (모델 훈련시 다커 컨테이너 경로), model_data 또한 이전 노트북에서 모델 훈련의 아티펙트의 경로인 train_model_artifact를 사용합니다.
- [알림] image_uri 를 여기서는 모델 훈련시의 다커 컨테이너와 동일하게 추론에서 사용했습니다. 추론 다커 컨테이너를 따로 구성해서 사용할 수도 있습니다.
- [알림] `TrainingStep`의 `properties` 속성은 [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) API의 응답결과와 동일합니다.


In [2]:
from sagemaker.model import Model

# sklearn_image_uri = '366743142698.dkr.ecr.ap-northeast-2.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3'

model = Model(
    image_uri=image_uri,
#     image_uri=sklearn_image_uri,    
    model_data= train_model_artifact,
    sagemaker_session=sagemaker_session,
    role=role,
)

In [3]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
)
step_create_model = CreateModelStep(
    name="FraudScratchModel",
    model=model,
    inputs=inputs,
)

### 파리마터, 단계, 조건을 조합하여 최종 파이프라인 정의



In [4]:
from sagemaker.workflow.pipeline import Pipeline


pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    steps=[step_create_model],
)

#### (선택) 파이프라인 정의 확인 

파이프라인을 정의하는 JSON을 생성하고 파이프라인 내에서 사용하는 파라미터와 단계별 속성들이 잘 정의되었는지 확인할 수 있습니다.

In [5]:
import json


definition = json.loads(pipeline.definition())
# definition

#### 파이프라인을 SageMaker에 제출하고 실행하기 

파이프라인 정의를 SageMaker Pipelines 서비스에 제출하여 파이프라인을 생성하거나 파이프라인이 이미 존재하면 파이프라인 정의를 업데이트합니다. 함께 전달되는 역할(role)을 이용하여 AWS에서 파이프라인을 생성하고 작업의 각 단계를 실행할 것입니다.   

In [6]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step',
 'ResponseMetadata': {'RequestId': 'e8e69e09-aaed-4481-8891-cf77a400e86c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e8e69e09-aaed-4481-8891-cf77a400e86c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Sun, 11 Jul 2021 03:16:04 GMT'},
  'RetryAttempts': 0}}

In [7]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [8]:
execution.describe()
execution.wait()

In [9]:
execution.list_steps()

[{'StepName': 'FraudScratchModel',
  'StartTime': datetime.datetime(2021, 7, 11, 3, 16, 5, 726000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 7, 11, 3, 16, 6, 552000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model/pipelines-im40b7hcs5s1-fraudscratchmodel-kydsghz1zj'}}}]

### SageMaker Studio에서 확인하기
- 이전의 3.1.Preprocesing-Pipleline 노트북에서 언급 되었듯이, SageMaker Studio 에서도 확인이 가능합니다. 이전 노트북을 참조 해주세요.


### 세이지 메이커 모델의 경로를 저장함. 
- 다음 노트북에서 사용합니다.

In [10]:
response = execution.list_steps()
sagemaker_model = response[0]['Metadata']['Model']['Arn'].split("/")[-1]
print("sagemaker_model: \n", sagemaker_model)

%store sagemaker_model

sagemaker_model: 
 pipelines-im40b7hcs5s1-fraudscratchmodel-kydsghz1zj
Stored 'sagemaker_model' (str)
